In [81]:
import requests
import pandas as pd
import numpy as np

In [39]:
def get_nasa(year, latlondf):
    json_list = []
    for i in latlondf.index:
        lat = latlondf.at[i,'lat']
        lon = latlondf.at[i,'lon']
        
        """
        Some Useful Parameters:
        - ALLSKY_SFC_SW_DWN: All Sky Insolation Incident on a Horizontal Surface (kwh/m2/day)
        - T2M: temperature at 2 meters from gound (C)
        - PRECTOT: precipitation (mm/day)
        - PS: Surface Pressure (kPa)
        - WS2M: Wind Speed at 2 Meters (m/s)
        - KT: Insolation Clearness Index (dimensionless)
        - DNR: Direct Normal Radiation (kWh/m^2/day)
        - DIFF: Diffuse Radiation On A Horizontal Surface (kWh/m^2/day)

        for more documentation: https://power.larc.nasa.gov/docs/v1/
        """

        parameters = 'ALLSKY_SFC_SW_DWN', 'KT', 'T2M', 'PRECTOT', 'PS', 'WS2M'
        parameters = ','.join(parameters)

        api_kwds = \
            {
                'user': 'anonymous',
                'parameters': parameters,
                'start_date': int(year),
                'end_date': int(year),
                'temp_average': 'INTERANNUAL',
                'lat': lat,
                'lon': lon
            }

        base = ('https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=SinglePoint&'
                'parameters={parameters}&'
                'startDate={start_date}&'
                'endDate={end_date}&'
                'userCommunity=SSE&'
                'tempAverage={temp_average}&'
                'outputList=JSON,ASCII&'
                'lat={lat}&'
                'lon={lon}&'
                'user={user}')

        url = base.format(**api_kwds)
        response = requests.get(url)
        json = response.json()
        json_list.append(json)
        
    return json_list

In [82]:
c = {'lat': [36, 36, 37, 37], 'lon': [45, 46, 47, 48]}
coords = pd.DataFrame(c)

In [83]:
coords

,lat,lon
0,36,45
1,36,46
2,37,47
3,37,48


In [84]:
jsons = get_nasa(2016, coords)

In [95]:
def join_coords(jsons):
    serieslist = []
    for i, el in enumerate(jsons):
        data = el['features'][0]['properties']['parameter']
        df = pd.DataFrame(data)
        series = df.iloc[-1,:]
        serieslist.append(series)

    attributes = pd.concat(serieslist, axis=1).T
    attributes.reset_index(drop=True, inplace=True)
    df = coords.join(attributes)
    return df

In [96]:
new_df = join_coords(jsons)

In [97]:
new_df

,lat,lon,ALLSKY_SFC_SW_DWN,KT,PRECTOT,PS,T2M,WS2M
0,36,45,4.91,0.58,1.17,86.38,14.18,2.18
1,36,46,5.03,0.60,1.00,83.20,12.18,2.71
2,37,47,4.80,0.58,0.77,82.31,10.39,3.09
3,37,48,4.44,0.54,1.83,84.02,9.98,2.39


,lat,lon,ALLSKY_SFC_SW_DWN,KT,PRECTOT,PS,T2M,WS2M
0,36,45,4.91,0.58,1.17,86.38,14.18,2.18
1,36,46,5.03,0.60,1.00,83.20,12.18,2.71
2,37,47,4.80,0.58,0.77,82.31,10.39,3.09
3,37,48,4.44,0.54,1.83,84.02,9.98,2.39


,lat,lon,ALLSKY_SFC_SW_DWN,KT,PRECTOT,PS,T2M,WS2M
0,36,45,4.91,0.58,1.17,86.38,14.18,2.18
1,36,46,5.03,0.60,1.00,83.20,12.18,2.71
2,37,47,4.80,0.58,0.77,82.31,10.39,3.09
3,37,48,4.44,0.54,1.83,84.02,9.98,2.39


In [46]:
df = pd.DataFrame(data)
df

,ALLSKY_SFC_SW_DWN,KT,PRECTOT,PS,T2M,WS2M
0,"{'201601': 2.2, '201602': 3.66, '201603': 3.57...","{'201601': 0.43, '201602': 0.58, '201603': 0.4...","{'201601': 2.43, '201602': 1.27, '201603': 3.3...","{'201601': 86.52, '201602': 86.72, '201603': 8...","{'201601': -0.26, '201602': 4.41, '201603': 6....","{'201601': 2.1, '201602': 1.83, '201603': 2.24..."
1,"{'201601': 2.63, '201602': 3.88, '201603': 4.0...","{'201601': 0.52, '201602': 0.62, '201603': 0.4...","{'201601': 1.91, '201602': 0.99, '201603': 2.7...","{'201601': 83.16, '201602': 83.44, '201603': 8...","{'201601': -1.61, '201602': 2.58, '201603': 4....","{'201601': 3.0, '201602': 2.32, '201603': 2.97..."
2,"{'201601': 2.6, '201602': 3.65, '201603': 3.94...","{'201601': 0.54, '201602': 0.6, '201603': 0.49...","{'201601': 0.95, '201602': 0.78, '201603': 1.4...","{'201601': 82.14, '201602': 82.49, '201603': 8...","{'201601': -1.81, '201602': 1.73, '201603': 4....","{'201601': 3.96, '201602': 3.03, '201603': 3.6..."
3,"{'201601': 2.42, '201602': 3.29, '201603': 3.7...","{'201601': 0.51, '201602': 0.54, '201603': 0.4...","{'201601': 1.64, '201602': 1.89, '201603': 1.8...","{'201601': 83.87, '201602': 84.23, '201603': 8...","{'201601': -1.31, '201602': 1.35, '201603': 5....","{'201601': 2.65, '201602': 2.09, '201603': 2.4..."


In [ ]:
df_annual = df.iloc[-1,:]

In [ ]:
arr = np.array({a:})